In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from glob import glob
import gc
import re
from scipy.io import wavfile
from scipy import signal


train_data_path = "data/train/train/audio/"
test_data_path = "data/test/test/audio/"

print(os.listdir("./"))

train_labels = 'yes no up down left right on off stop go silence unknown'.split()

['get_data.py', 'data-preprocessing.ipynb', 'kaggle_0.csv', 'EDA.ipynb', 'testing-kaggle-submission-conv-and-lstm.ipynb', 'testing-kaggle-submission.ipynb', 'data', '.git', 'kaggle_2.csv', 'kaggle_3.csv', '.ipynb_checkpoints', 'README.md', 'models', 'results', 'kaggle_gru_1.csv', 'testing-kaggle-submission-Recurrent.ipynb', 'test-json.json', '.gitignore', 'kaggle_lstm_1.csv', 'deep_learning_methods_project_3.ipynb', 'kaggle_4.csv', 'architectures-benchmark.ipynb', 'data-preprocessing-Copy1.ipynb']


In [2]:
def log_specgram(audio, sample_rate, window_size=20,

                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)

In [3]:
def list_wavs_fname(dirpath, ext='wav'):
    print(dirpath)
    fpaths = glob(os.path.join(dirpath, r'*/*' + ext))
    pat = r'.+/(\w+)/\w+\.' + ext + '$'
    labels = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            labels.append(r.group(1))
    pat = r'.+/(\w+\.' + ext + ')$'
    fnames = []
    for fpath in fpaths:
        r = re.match(pat, fpath)
        if r:
            fnames.append(r.group(1))
    return labels, fnames

In [4]:
# pad_audio will pad audios that are less than 16000(1 second) with 0s to make them all have the same length.

# chop_audio will chop audios that are larger than 16000(eg. wav files in background noises folder) to 16000 in length. Inaddition, it will create several chunks out of one large wav files given the parameter 'num'.

# label_transform transform labels into dummies values. It's used in combination with softmax to predict the label.


In [5]:
def pad_audio(samples):
    if len(samples) >= L: return samples
    else: return np.pad(samples, pad_width=(L - len(samples), 0), mode='constant', constant_values=(0, 0))


In [6]:
def chop_audio(samples, L=16000, num=20):
    for i in range(num):
        beg = np.random.randint(0, len(samples) - L)
        yield samples[beg: beg + L]


In [7]:
def label_transform(labels):
    nlabels = []
    for label in labels:
        if label == '_background_noise_':
            nlabels.append('silence')
        elif label not in train_labels:
            nlabels.append('unknown')
        else:
            nlabels.append(label)
    return pd.get_dummies(pd.Series(nlabels))


In [9]:
labels, fnames = list_wavs_fname(train_data_path)

L = 16000
new_sample_rate = 8000
Y_train = []
X_train = []

for label, fname in zip(labels, fnames):
    sample_rate, samples = wavfile.read(os.path.join(train_data_path, label, fname))
    samples = pad_audio(samples)
    if len(samples) > 16000:
        n_samples = chop_audio(samples)
    else: n_samples = [samples]
    for samples in n_samples:
        resampled = signal.resample(samples, int(new_sample_rate / sample_rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        Y_train.append(label)
        X_train.append(specgram)
X_train = np.array(X_train)
X_train = X_train.reshape(tuple(list(X_train.shape) + [1]))
Y_train = label_transform(Y_train)
label_index = Y_train.columns.values
Y_train = Y_train.values
Y_train = np.array(Y_train)

del labels, fnames
gc.collect()

print(X_train.shape)


data/train/train/audio/


/home/krzys_spalinski/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: WavFileWarning: Chunk (non-data) not understood, skipping it.
  if __name__ == '__main__':


(64841, 99, 81, 1)


In [10]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [31]:
input_shape = (99, 81, 1)
nclass = 12
inp = Input(shape=input_shape)
norm_inp = BatchNormalization()(inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
img_1 = Dropout(rate=0.2)(img_1)
img_1 = Flatten()(img_1)

dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

model = models.Model(inputs=inp, outputs=dense_1)
opt = optimizers.Adam()

model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 99, 81, 1)]       0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 99, 81, 1)         4         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 98, 80, 8)         40        
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 97, 79, 8)         264       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 96, 78, 8)         264       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 39, 8)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 48, 39, 8)         0   

In [33]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size=0.1, random_state=2017)
history = model.fit(x_train, y_train, batch_size=32, validation_data=(x_valid, y_valid), epochs=1, shuffle=True)

#model.save(os.path.join(model_path, 'cnn.model'))




1642/1642 [==============================] - 154s 94ms/step - loss: 0.1550 - accuracy: 0.6595 - val_loss: 0.0905 - val_accuracy: 0.7856


In [ ]:
import json

In [39]:
json.dumps(history.history)

'{"loss": [0.15503691136837006], "accuracy": [0.6595011353492737], "val_loss": [0.09053700417280197], "val_accuracy": [0.7856408357620239]}'

In [12]:
def test_data_generator(batch=16):
    fpaths = glob(os.path.join(test_data_path, '*wav'))
    i = 0
    for path in fpaths:
        if i == 0:
            imgs = []
            fnames = []
        i += 1
        rate, samples = wavfile.read(path)
        samples = pad_audio(samples)
        resampled = signal.resample(samples, int(new_sample_rate / rate * samples.shape[0]))
        _, _, specgram = log_specgram(resampled, sample_rate=new_sample_rate)
        imgs.append(specgram)
        fnames.append(path.split('\\')[-1])
        if i == batch:
            i = 0
            imgs = np.array(imgs)
            imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
            yield fnames, imgs
    if i < batch:
        imgs = np.array(imgs)
        imgs = imgs.reshape(tuple(list(imgs.shape) + [1]))
        yield fnames, imgs
    return

# del x_train, y_train
# gc.collect()

# index = []
# results = []

# print("Test")

# for fnames, imgs in tqdm(test_data_generator(batch=32)):
#     predicts = model.predict(imgs)
#     predicts = np.argmax(predicts, axis=1)
#     predicts = [label_index[p] for p in predicts]
#     index.extend([text.split("/")[-1] for text in fnames])
#     results.extend(predicts)

# df = pd.DataFrame(columns=['fname', 'label'])
# df['fname'] = index
# df['label'] = results
# df.to_csv('sub.csv', index=False)

In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint
import json

def reset_weights(model):
    for layer in model.layers: 
        if hasattr(layer, 'kernel_initializer'):
            layer.kernel.initializer.run()
            
def validate_architecture(architecture_name,
                          num_epochs,
                          num_iterations):
    
    for it in range(num_iterations):
    
        # Reinitializing  weights
        #reset_weights(model)
        # no idea how to reset weights
        input_shape = (99, 81, 1)
        nclass = len(train_labels)
        inp = Input(shape=input_shape)
        norm_inp = BatchNormalization()(inp)
        img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(norm_inp)
        img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
        img_1 = Convolution2D(8, kernel_size=2, activation=activations.relu)(img_1)
        img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
        img_1 = Dropout(rate=0.2)(img_1)
        img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
        img_1 = Convolution2D(16, kernel_size=3, activation=activations.relu)(img_1)
        img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
        img_1 = Dropout(rate=0.2)(img_1)
        img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
        img_1 = Convolution2D(32, kernel_size=3, activation=activations.relu)(img_1)
        img_1 = MaxPooling2D(pool_size=(2, 2))(img_1)
        img_1 = Dropout(rate=0.2)(img_1)
        img_1 = Flatten()(img_1)

        dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(img_1))
        dense_1 = BatchNormalization()(Dense(128, activation=activations.relu)(dense_1))
        dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

        model = models.Model(inputs=inp, outputs=dense_1)
        opt = optimizers.Adam()

        model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['accuracy'])
    
        x_train, x_valid, y_train, y_valid = train_test_split(X_train,
                                                              Y_train,
                                                              test_size=0.1)
        
        mc = ModelCheckpoint('best_model.h5',
                     monitor='val_accuracy',
                     mode='max',
                     verbose=1,
                     save_weights_only=True,
                     save_best_only=True)
        
        history = model.fit(x_train,
                            y_train,
                            batch_size=32,
                            validation_data=(x_valid, y_valid),
                            epochs=num_epochs,
                            shuffle=True,
                            callbacks=[mc])
        
        history = history.history
        
        # saving json
        with open(f'./results/results_{architecture_name}_{it}', 'w') as fp:
            json.dump(history, fp, indent=4)
            
            
        # prediction
        index = []
        results = []
        
        for fnames, imgs in tqdm(test_data_generator(batch=32)):
            predicts = model.predict(imgs)
            predicts = np.argmax(predicts, axis=1)
            predicts = [label_index[p] for p in predicts]
            index.extend([text.split("/")[-1] for text in fnames])
            results.extend(predicts)

            df = pd.DataFrame(columns=['fname', 'label'])
            df['fname'] = index
            df['label'] = results
            df.to_csv(f'./results/submission_{architecture_name}_{it}.csv', index=False)

In [14]:
validate_architecture("baseline_test", 1, 1)

1824/1824 [==============================] - ETA: 0s - loss: 0.1265 - accuracy: 0.7026
Epoch 00001: val_accuracy improved from -inf to 0.84518, saving model to best_model.h5
1824/1824 [==============================] - 177s 97ms/step - loss: 0.1265 - accuracy: 0.7026 - val_loss: 0.0662 - val_accuracy: 0.8452


4955it [21:46,  3.79it/s]
